In [19]:
import pandas as pd

In [24]:
df_matriculas = pd.DataFrame()

# importando notas de 2010 a 2017
for i in range(10,18):
    for j in range(1,3):
        print('20'+str(i)+str(j))
        df_matricula = pd.read_csv('../matriculas_new/matricula-componente-20'+str(i)+str(j)+'.csv',sep=';')
        #concatena as tabelas de 2010 a 2017
        df_matriculas = pd.concat([df_matriculas, df_matricula])

20101
20102
20111
20112
20121
20122
20131
20132
20141
20142
20151


/home/andressa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  if __name__ == '__main__':


20152
20161
20162
20171
20172


In [54]:
cols = ['descricao', 'discente', 'faltas_unidade', 'id_curso',
       'id_turma', 'media_final', 'nota', 'numero_total_faltas', 'reposicao',
       'unidade']
df_matriculas = df_matriculas[cols]

In [55]:
# obtendo o id do curso de ciências e tecnologia
df_cursos = pd.read_csv('../cursos-graduacao.csv', sep=';')
id_cet = df_cursos[df_cursos.nome == 'CIÊNCIAS E TECNOLOGIA'].id_curso

print("Id Cet: "+ str(id_cet))

Id Cet: 16    10320810
Name: id_curso, dtype: int64


In [56]:
# filtra tabela pelo id_curso
df_matriculas = df_matriculas[df_matriculas.id_curso == float(id_cet)]

In [57]:
# concatenar todas as turmas de 2010 a 2017
df_turmas = pd.DataFrame()
for i in range(10,18):
    for j in range(1,3):
        print(str(i)+str(j))
        f = open('../turmas_new/turmas-20'+str(i)+str(j)+'.csv')
        d = f.read()
        d = d.replace("\n","")
        x = d.split("\";\"")
        a = []
        step = 25
        for b in range(0, len(x), step):
            a.append(x[b:b+step])
        df = pd.DataFrame(a[1:],columns=['id_turma']+a[0][1:])
        df_turmas = pd.concat([df_turmas, df])

101
102
111
112
121
122
131
132
141
142
151
152
161
162
171
172


In [77]:
# filtra disciplinas da graduação
df_turmas = df_turmas[df_turmas['nivel_ensino'] == 'GRADUAÇÃO']

In [78]:
# conversoes
df_turmas['id_turma'] = df_turmas.id_turma.astype(int)
df_turmas['id_componente_curricular'] = df_turmas.id_componente_curricular.astype(int)
df_turmas['ano'] = df_turmas.ano.astype(int)
df_turmas['periodo'] = df_turmas.periodo.astype(int)

In [79]:
# nova coluna
series_anoperiodo = (df_turmas['ano'].map(str)+df_turmas['periodo'].map(str))
df_turmas['periodoano'] = series_anoperiodo.astype(int)

# filtrando por ano e nivel de ensino
df_turmasFilt = df_turmas[(df_turmas['ano'] >= 2015) & 
                          (df_turmas['nivel_ensino'] == 'GRADUAÇÃO')]

In [80]:
df_componentes = pd.read_csv('../componentes-curriculares-presenciais-ok.csv')

df_turmasNome = pd.merge(df_turmasFilt, df_componentes, left_on='id_componente_curricular', right_on='id_componente')

In [81]:
colunas = ['id_turma', 'codigo', 'nome', 'pre_requisito', 'ano', 'periodo', 'unidade_responsavel', 'periodoano']
df_turmasNome = df_turmasNome[colunas]

In [82]:
df_auxAll = pd.merge(df_matriculas, df_turmasNome, on='id_turma')
df_auxAll = df_auxAll.drop_duplicates().dropna()

In [83]:
# conjunto de alunos que não serão analisados
series_Antes2015 = df_auxAll[df_auxAll['ano']<2015].discente

# selecionando apenas alunos que entraram a partir de 2015
df_depois2015 = df_auxAll[~df_auxAll['discente'].isin(series_Antes2015)]
df_depois2015 = df_depois2015[df_depois2015['unidade_responsavel'] == 'ESCOLA DE CIÊNCIAS E TECNOLOGIA']

disciplinas = ['CÁLCULO I','CÁLCULO II','CÁLCULO III','VETORES E GEOMETRIA ANALÍTICA','PRÉ-CÁLCULO','PROBABILIDADE E ESTATÍSTICA',
               'INTRODUÇÃO À FÍSICA CLÁSSICA I','INTRODUÇÃO À FÍSICA CLÁSSICA II','INTRODUÇÃO À FÍSICA CLÁSSICA III','LÓGICA DE PROGRAMAÇÃO',
               'LINGUAGEM DE PROGRAMAÇÃO','COMPUTAÇÃO NUMÉRICA','QUÍMICA GERAL','ÁLGEBRA LINEAR','MECÂNICA DOS SÓLIDOS','MECÂNICA DOS FLUIDOS',
               'CIÊNCIA E TECNOLOGIA DOS MATERIAIS']

df_auxTurmas = df_depois2015[df_depois2015['nome'].isin(disciplinas)]

colunas_3 = ['descricao', 'discente', 'id_turma', 'media_final', 'codigo',
       'nome', 'ano', 'periodo', 'unidade_responsavel', 'id_curso','periodoano']
df_auxTurmas = df_auxTurmas[colunas_3]
df_auxTurmas.to_csv("../turmas_2015-2017.csv")